# Train Recommender with Apache Spark ML
Next, use the Amazon SageMaker Python SDK to submit a processing job. Use the Spark container that was just built with our Spark script.

In [1]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

In [2]:
s3_input_data = 's3://sagemaker-us-east-1-835319576252/spark_datasets/train/movielens/sample_movielens_ratings.txt'

print(s3_input_data)

s3://sagemaker-us-east-1-835319576252/spark_datasets/train/movielens/sample_movielens_ratings.txt


In [3]:
!aws s3 ls $s3_input_data

2020-11-02 20:04:44      32363 sample_movielens_ratings.txt


## Setup Output Data

In [4]:
from time import gmtime, strftime
from time import gmtime, strftime
timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

output_prefix = 'spark-als-{}'.format(timestamp_prefix)
processing_job_name = 'spark-als-{}'.format(timestamp_prefix)

print('Processing job name:  {}'.format(processing_job_name))

Processing job name:  spark-als-2020-11-03-07-05-11


In [5]:
s3_output_data = 's3://{}/{}/output'.format(bucket, output_prefix)

print(s3_output_data)

s3://sagemaker-us-east-1-835319576252/spark-als-2020-11-03-07-05-11/output


# Review the Spark preprocessing script.

In [6]:
!pygmentize /root/workshop/02_usecases/sagemaker_recommendations/src/train_spark_als.py

from __future__ import print_function
from __future__ import unicode_literals

import time
import sys
import os
import shutil
import csv

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row


def main():
    args_iter = iter(sys.argv[1:])
    args = dict(zip(args_iter, args_iter))
    
    # Retrieve the args and replace 's3://' with 's3a://' (used by Spark)
    s3_input_data = args['s3_input_data'].replace('s3://', 's3a://')
    print(s3_input_data)
    s3_output_data = args['s3_output_data'].replace('s3://', 's3a://')
    print(s3_output_data)
    
    spark = SparkSession.builder \
        .appName("Spark_ALS") \
        .getOrCreate()

    lines = spark.read.text(s3_input_data).rdd
    parts = lines.map(lambda row: row.value.split("::"))
    ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), 
                            

In [7]:
from sagemaker.spark.processing import PySparkProcessor

processor = PySparkProcessor(base_job_name='spark-als',
                             role=role,
                             instance_count=1,
                             instance_type='ml.r5.2xlarge',
                             max_runtime_in_seconds=1200)

## Start the Spark Processing Job

_Notes on not using ProcessingInput and Output:_
* Since Spark natively reads/writes from/to S3 using s3a://, we can avoid the copy required by ProcessingInput and ProcessingOutput (FullyReplicated or ShardedByS3Key) and just specify the S3 input and output buckets/prefixes._"
* See https://github.com/awslabs/amazon-sagemaker-examples/issues/994 for issues related to using /opt/ml/processing/input/ and output/
* If we use ProcessingInput, the data will be copied to each node (which we don't want in this case since Spark already handles this)

In [8]:
from sagemaker.processing import ProcessingOutput

processor.run(submit_app='/root/workshop/02_usecases/sagemaker_recommendations/src/train_spark_als.py',
              arguments=['s3_input_data', s3_input_data,
                         's3_output_data', s3_output_data,
              ],
              logs=True,
              wait=False
)


Job Name:  spark-als-2020-11-03-07-05-12-360
Inputs:  [{'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-835319576252/spark-als-2020-11-03-07-05-12-360/input/code/train_spark_als.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  []


In [9]:
from IPython.core.display import display, HTML

processing_job_name = processor.jobs[-1].describe()['ProcessingJobName']

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/{}">Processing Job</a></b>'.format(region, processing_job_name)))


In [10]:
from IPython.core.display import display, HTML

processing_job_name = processor.jobs[-1].describe()['ProcessingJobName']

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After a Few Minutes</b>'.format(region, processing_job_name)))


In [11]:
from IPython.core.display import display, HTML

s3_job_output_prefix = output_prefix

display(HTML('<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Spark Job Has Completed</b>'.format(bucket, s3_job_output_prefix, region)))


# Monitor the Processing Job

In [12]:
running_processor = sagemaker.processing.ProcessingJob.from_processing_name(processing_job_name=processing_job_name,
                                                                            sagemaker_session=sagemaker_session)

processing_job_description = running_processor.describe()

print(processing_job_description)

{'ProcessingInputs': [{'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-835319576252/spark-als-2020-11-03-07-05-12-360/input/code/train_spark_als.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}], 'ProcessingJobName': 'spark-als-2020-11-03-07-05-12-360', 'ProcessingResources': {'ClusterConfig': {'InstanceCount': 1, 'InstanceType': 'ml.r5.2xlarge', 'VolumeSizeInGB': 30}}, 'StoppingCondition': {'MaxRuntimeInSeconds': 1200}, 'AppSpecification': {'ImageUri': '173754725891.dkr.ecr.us-east-1.amazonaws.com/sagemaker-spark-processing:2.4-cpu', 'ContainerEntrypoint': ['smspark-submit', '/opt/ml/processing/input/code/train_spark_als.py'], 'ContainerArguments': ['s3_input_data', 's3://sagemaker-us-east-1-835319576252/spark_datasets/train/movielens/sample_movielens_ratings.txt', 's3_output_data', 's3://sagemaker-us-east-1-835319576252/spark-als-2020-11-03

In [13]:
running_processor.wait()

........................11-03 07:08 smspark.cli  INFO     Parsing arguments. argv: ['/usr/local/bin/smspark-submit', '/opt/ml/processing/input/code/train_spark_als.py', 's3_input_data', 's3://sagemaker-us-east-1-835319576252/spark_datasets/train/movielens/sample_movielens_ratings.txt', 's3_output_data', 's3://sagemaker-us-east-1-835319576252/spark-als-2020-11-03-07-05-11/output']
11-03 07:08 smspark.cli  INFO     Raw spark options before processing: {'class_': None, 'jars': None, 'py_files': None, 'files': None, 'verbose': False}
11-03 07:08 smspark.cli  INFO     App and app arguments: ['/opt/ml/processing/input/code/train_spark_als.py', 's3_input_data', 's3://sagemaker-us-east-1-835319576252/spark_datasets/train/movielens/sample_movielens_ratings.txt', 's3_output_data', 's3://sagemaker-us-east-1-835319576252/spark-als-2020-11-03-07-05-11/output']
11-03 07:08 smspark.cli  INFO     Rendered spark options: {'class_': None, 'jars': None, 'py_files': None, 'files': None, 'verbose': False}


# _Please Wait Until the ^^ Processing Job ^^ Completes Above._

# Inspect the Processed Output 

## These are the quality checks on our dataset.

## _The next cells will not work properly until the job completes above._

In [14]:
!aws s3 ls --recursive $s3_output_data/

## TODO: Copy the Output from S3 to Local

In [15]:
!aws s3 cp --recursive $s3_output_data ./spark-als-output/ --exclude="*" --include="*.csv"

## Review Recommendations

In [16]:
import glob
import pandas as pd
import os

def load_dataset(path, sep, header):
    data = pd.concat([pd.read_csv(f, sep=sep, header=header) for f in glob.glob('{}/*.csv'.format(path))], ignore_index = True)

    return data

In [17]:
df_recommendations = load_dataset(path='./spark-als-output/recommendations/', sep='\t', header=0)
df_recommendations

ValueError: No objects to concatenate

# Save for the Next Notebook(s)

In [ ]:
# %%javascript
# Jupyter.notebook.save_checkpoint();
# Jupyter.notebook.session.delete();